# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
from pandas import DataFrame
import os
import json

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_info = "city_info2.csv"

city_info = pd.read_csv(city_info)
print(city_info.head())
#print(city_info.info)

   Unnamed: 0      City  Latitude  Longitude  Temperature  Humidity  \
0           0   Rikitea    -23.12    -134.97        77.45        69   
1           1    Albany     42.60     -73.97        38.05        46   
2           2  Frontera     26.93    -101.45        79.00        60   
3           3   Oparino     59.85      48.28        31.77        90   
4           4    Luanda     -8.84      13.23        77.00        94   

   Cloudiness  Wind Speed  
0          71        7.36  
1           1        1.01  
2          61        1.99  
3          90        3.91  
4          75        3.36  


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [4]:
location = city_info[["Latitude","Longitude"]]
humidity = city_info["Humidity"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=humidity, dissipating=False, max_intensity=10,
                                 #point_radius=1)
fig.add_layer(heat_layer)
fig
#This was working until i re-ran everything

SyntaxError: invalid syntax (<ipython-input-4-ff99b5f879f6>, line 7)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
city_info.describe()

,Unnamed: 0,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
count,1241.000000,1241.000000,1241.000000,1241.000000,1241.000000,1241.000000,1241.000000
mean,620.000000,22.193723,22.170685,59.731120,67.717969,50.653505,7.237309
std,358.390151,30.848780,85.506102,20.163654,21.396889,39.057536,5.018580
min,0.000000,-54.800000,-179.170000,-4.000000,7.000000,0.000000,0.040000
25%,310.000000,-2.890000,-54.110000,44.800000,56.000000,9.000000,3.360000
50%,620.000000,26.770000,27.360000,61.840000,72.000000,47.000000,6.260000
75%,930.000000,47.800000,96.110000,76.910000,83.000000,90.000000,9.660000
max,1240.000000,78.220000,179.320000,96.800000,100.000000,100.000000,33.550000


In [6]:
# * Narrow down the DataFrame to find your ideal weather condition. For example:
#   * A max temperature lower than 80 degrees but higher than 70.
#   * Wind speed less than 10 mph.
#   * Zero cloudiness.
#   * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
#temp_df = city_info.loc[(city_info["Temperature"]<80)&(city_info["Temperature"]>70)]&[(city_info["Wind Speed"]<10)]&[(city_info["Cloudiness"]==0)]

temp_df = city_info.loc[(city_info["Temperature"]<80)&(city_info["Temperature"]>70)]
temp_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,0,Rikitea,-23.12,-134.97,77.45,69,71,7.36
2,2,Frontera,26.93,-101.45,79.00,60,61,1.99
4,4,Luanda,-8.84,13.23,77.00,94,75,3.36
8,8,Grand Gaube,-20.01,57.66,77.00,78,40,13.87
22,22,Bilma,18.69,12.92,79.16,15,33,5.84
...,...,...,...,...,...,...,...,...
1201,1201,Sassandra,4.95,-6.08,77.61,89,100,6.31
1210,1210,Gouré,13.98,10.27,76.17,15,3,7.40
1216,1216,Hirara,24.80,125.28,78.80,83,75,20.80
1226,1226,Mlalo,-4.93,38.93,72.97,88,100,8.59


In [7]:
# * Narrow down the DataFrame to find your ideal weather condition. For example:
#   * A max temperature lower than 80 degrees but higher than 70.
#   * Wind speed less than 10 mph.
#   * Zero cloudiness.
#   * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
wind_df = temp_df.loc[(city_info["Wind Speed"]<10)]
wind_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,0,Rikitea,-23.12,-134.97,77.45,69,71,7.36
2,2,Frontera,26.93,-101.45,79.00,60,61,1.99
4,4,Luanda,-8.84,13.23,77.00,94,75,3.36
22,22,Bilma,18.69,12.92,79.16,15,33,5.84
23,23,Mattru,7.62,-11.83,76.10,87,100,2.21
...,...,...,...,...,...,...,...,...
1156,1156,Kisangani,0.52,25.20,70.34,86,99,1.74
1165,1165,Cienfuegos,22.15,-80.44,78.39,88,33,2.24
1201,1201,Sassandra,4.95,-6.08,77.61,89,100,6.31
1210,1210,Gouré,13.98,10.27,76.17,15,3,7.40


In [8]:
beast_weather_ever_df = temp_df.loc[(city_info["Cloudiness"]==0)]
beast_weather_ever_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
75,75,Panabá,21.28,-88.27,79.83,66,0,10.65
281,281,Bouza,14.42,6.04,77.18,14,0,6.58
362,362,Carupano,10.67,-63.26,74.30,85,0,6.82
398,398,Madaoua,14.07,5.96,78.35,14,0,7.11
481,481,Rāwah,34.48,41.91,70.63,34,0,5.55
540,540,Tateyama,34.98,139.87,73.99,36,0,8.99
650,650,Alta Floresta,-9.88,-56.09,70.59,89,0,3.47
672,672,Maturin,9.75,-63.18,75.11,73,0,5.68
1136,1136,Lianran,24.92,102.48,72.59,40,0,20.13


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#add column for hotel name 
hotel_df = beast_weather_ever_df.copy()
hotel_df['Hotel Name'] = ""
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
75,75,Panabá,21.28,-88.27,79.83,66,0,10.65,
281,281,Bouza,14.42,6.04,77.18,14,0,6.58,
362,362,Carupano,10.67,-63.26,74.30,85,0,6.82,
398,398,Madaoua,14.07,5.96,78.35,14,0,7.11,
481,481,Rāwah,34.48,41.91,70.63,34,0,5.55,
540,540,Tateyama,34.98,139.87,73.99,36,0,8.99,
650,650,Alta Floresta,-9.88,-56.09,70.59,89,0,3.47,
672,672,Maturin,9.75,-63.18,75.11,73,0,5.68,
1136,1136,Lianran,24.92,102.48,72.59,40,0,20.13,


In [10]:
# #Set parameters to search for hotels with 5000 meters.
# #Store the first Hotel result into the DataFrame.
# set up a parameters dictionary
params = {
    "location" : f'{lat},{lng}',
    "radius" : 5000,
    "types" : "lodging",
    "keyword" : "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f'{lat},{lng}'
# #Hit the Google Places API for each city's coordinates.
# # base url    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# run a request using our params dictionary
    names = requests.get(base_url, params=params)
    hotel_name = names.json()
    #print(json.dumps(response, indent=4, sort_keys=True))
    try:
        hotel_df.loc[index, "Hotel Name"]= hotel_name["results"][0]['name']
    except (KeyError, IndexError):
        print("No Hotel Found!!!")
hotel_df

NameError: name 'lat' is not defined

In [ ]:
#print the response url, avoid doing for public github repos in order to avoid exposing key


In [ ]:
# #Plot markers on top of the heatmap.

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_df = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

KeyError: 'Country'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
